# Setup

In [1]:
import os, re, json, math, random, hashlib, time
import numpy as np, pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
import spacy

nlp = spacy.load("en_core_web_sm")

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

DATA_DIR = "data/"


# ML Model Training

In [2]:
import pandas as pd
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.metrics import f1_score, accuracy_score
from torch import nn  # <-- Added for class-weighted loss
import torch.nn.functional as F # For Focal CE Loss

from sklearn.model_selection import train_test_split
from sklearn.utils import resample # for over-sampling

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha=alpha
        self.gamma=gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction="none")
        pt = torch.exp(-ce_loss)
        loss = (1 - pt) ** self.gamma * ce_loss
        if self.alpha is not None: 
            loss *= self.alpha[targets]
        return loss.mean()


df_clean = pd.read_json(r'data\ground_truth-2.json', lines=True)
print(df_clean.head())

# map labels to numbers
label_map = {"Advertisement":0, "Irrelevant Content":1,
             "Rant without visiting":2, "None":3}

inv_label_map = {v:k for k,v in label_map.items()}

# Map labels to integers
df_clean["labels"] = df_clean["GPT-label"].map(label_map)

# Drop rows where mapping failed (NaN)
df_clean = df_clean.dropna(subset=["labels"])

# Convert labels to integer
df_clean["labels"] = df_clean["labels"].astype(int)

df_clean = df_clean.copy()
df_clean["text_clean"] = df_clean["text_clean"].astype(str).fillna("")
df_clean["text_for_model"] = df_clean["text_clean"] #+ " [SENTIMENT: " + df_clean["sentiment_label"] + "]"
df_clean["text_for_model"] = df_clean["text_for_model"].astype(str).fillna("")

df_clean["text_for_model"] = (
    df_clean["text_clean"].astype(str) +
    #" [SENTIMENT: " + df_clean["sentiment_label"].astype(str) + "]" +
    " [RULE_AD: " + df_clean["rule_advertisement"].fillna(False).astype(str) + "]" +
    " [RULE_IRR: " + df_clean["rule_irrelevant"].fillna(False).astype(str) + "]" +
    " [RULE_RANT: " + df_clean["rule_rant_without_visit"].fillna(False).astype(str) + "]"
)

train_df, test_df = train_test_split(df_clean, test_size=0.1, stratify=df_clean["labels"])


        user_id               name                                gmap_id  \
0  1.130049e+20              Ben A  0x8626a5074449db49:0x79fa4fa238746342   
1  1.088190e+20       michele vess  0x8626baf396726b1f:0xd33145fb706bec2d   
2  1.047214e+20  Danyelle Williams  0x8631c517e81069d3:0x720670c8e8f84fc3   
3  1.140741e+20           CK Moody  0x86249e700409f4af:0x204bb6ea088067c2   
4  1.013455e+20              Ant01   0x889de6c064ee6423:0x5b69e22b1279b2b   

   rating                                           text_raw  \
0       1  It's a nice enough McDonalds, but I can't stan...   
1       4  Really great street tacos tons of sauces for t...   
2       4               Went to visit someone place was nicw   
3       5  Great store to visit with a large variety of p...   
4       5  Excellent! Very knowledgeable sales manager.  ...   

                                          text_clean           time  \
0  nice mcdonalds cant stand self order kiosk obv...  1547600291921   
1  great 

In [3]:
# Oversampling
dfs = [train_df[train_df['labels'] == i] for i in train_df['labels'].unique()]
max_size = max(len(df) for df in dfs)

dfs_upsampled = [resample(df, replace=True, n_samples=max_size) for df in dfs]
train_df = pd.concat(dfs_upsampled).sample(frac=1)

# hugging face dataset
# dataset = Dataset.from_pandas(df_clean[["text_for_model", "labels"]])
train_dataset = Dataset.from_pandas(train_df[["text_for_model", "labels"]]) # over-sample
test_dataset = Dataset.from_pandas(test_df[["text_for_model", "labels"]])
# dataset = dataset.train_test_split(test_size=0.1) # only if splitting after conversion

# Tokenize
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text_for_model"], truncation=True, padding="max_length", max_length=256)



c:\Users\estelle\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
print(train_dataset)

# dataset = dataset.map(tokenize, batched=True)
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Dataset({
    features: ['text_for_model', 'labels', '__index_level_0__'],
    num_rows: 33424
})


Map:   0%|          | 0/33424 [00:00<?, ? examples/s]

Map:   0%|          | 0/1058 [00:00<?, ? examples/s]

In [5]:
# Compute class weights   <-- NEW SECTION
labels = df_clean["labels"]
class_counts = labels.value_counts().sort_index()
class_weights = 1.0 / class_counts
class_weights = torch.tensor(class_weights.values, dtype=torch.float32).to("cuda" if torch.cuda.is_available() else "cpu")
class_weights /= class_weights.sum()

# Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,
    ignore_mismatched_sizes=True,
    hidden_dropout_prob = 0.1,
    attention_probs_dropout_prob = 0.2
)

# Define custom loss function  <-- NEW SECTION
#loss_fct = nn.CrossEntropyLoss(weight=class_weights)
loss_fct = FocalLoss(gamma=2)

def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    loss = loss_fct(outputs.logits, labels)
    return (loss, outputs) if return_outputs else loss

# Training Setup
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=25, save_steps=25,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,  # <-- Increased for better learning
    weight_decay=1e-2,
    logging_dir="./logs",
    logging_steps=50,
    warmup_ratio=0.01
)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=1)
    labels = torch.tensor(labels)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }

# Subclass Trainer to use weighted loss  <-- NEW SECTION
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = loss_fct(outputs.logits, labels)  # Focal loss
        return (loss, outputs) if return_outputs else loss

# Initialize trainer using subclass  <-- NEW SECTION
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # optional, for monitoring metrics
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\estelle\AppData\Local\Temp\ipykernel_25656\3740128401.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: estelle-sim (estelle

Step,Training Loss,Validation Loss,Accuracy,F1 Macro
25,No log,0.773462,0.054820,0.025986
50,0.761300,0.580902,0.753308,0.314117
75,0.761300,0.493919,0.747637,0.377855
100,0.564400,0.457019,0.675803,0.377810
125,0.564400,0.446140,0.613422,0.398451
150,0.400300,0.385412,0.682420,0.442362
175,0.400300,0.383598,0.689981,0.456425
200,0.288400,0.318394,0.721172,0.474440
225,0.288400,0.196361,0.843100,0.554689
250,0.213300,0.297090,0.760870,0.492031


KeyboardInterrupt: 

In [13]:
model = model.from_pretrained(r"results\best")
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # optional, for monitoring metrics
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.evaluate()

C:\Users\estelle\AppData\Local\Temp\ipykernel_25656\4292709683.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


{'eval_loss': 0.15624478459358215,
 'eval_model_preparation_time': 0.001,
 'eval_accuracy': 0.8610586011342155,
 'eval_f1_macro': 0.5893328661136195,
 'eval_runtime': 3.3553,
 'eval_samples_per_second': 315.323,
 'eval_steps_per_second': 10.133}

In [ ]:
import numpy as np

def get_probs(model, tokenizer, texts, device="cuda"):
    model.eval()
    probs = []
    with torch.no_grad():
        for batch in np.array_split(texts, 32):  # mini-batching
            enc = tokenizer(list(batch), return_tensors="pt", truncation=True, padding="max_length", max_length=256).to(device)
            logits = model(**enc).logits
            p = torch.softmax(logits, dim=1).cpu().numpy()
            probs.append(p)
    return np.vstack(probs)

def tune_thresholds(y_true, probs, classes=[0,1,2], none_class=3):
    best_f1, best_thresh = 0, None

    # search thresholds between 0.1 and 0.9
    search_space = np.linspace(0.1, 0.9, 17)

    for t0 in search_space:
        for t1 in search_space:
            for t2 in search_space:
                thresholds = [t0, t1, t2]
                preds = []

                for p in probs:
                    # check classes 0,1,2 in order
                    chosen = none_class
                    for c, t in zip(classes, thresholds):
                        if p[c] >= t and p[c] == max(p[classes]):  
                            chosen = c
                            break
                    preds.append(chosen)

                f1 = f1_score(y_true, preds, average="macro")
                if f1 > best_f1:
                    best_f1, best_thresh = f1, thresholds

    return best_f1, best_thresh

probs = get_probs(model, tokenizer, test_df['text_for_model'].tolist())
best_f1, best_thresh = tune_thresholds(test_df['labels'].values, probs)
print("Best F1:", best_f1, "with thresholds:", best_thresh)

Best F1: 0.6077086077297706 with thresholds: [0.5, 0.55, 0.55]


In [ ]:
import optuna
from sklearn.metrics import f1_score

def objective(trial, probs_val, y_val):
    # per-class thresholds for 0..2
    t0 = trial.suggest_float("t0", 0.01, 0.6)
    t1 = trial.suggest_float("t1", 0.01, 0.6)
    t2 = trial.suggest_float("t2", 0.01, 0.6)
    t_none = trial.suggest_float("t_none", 0.01, 0.9)
    delta = trial.suggest_float("delta", 0.0, 0.4)

    # temperature (if you provide logits instead of probs you can rescale first)
    preds = []
    for p in probs_val:
        # use margin rule
        top = p.argmax()
        runner_up = np.argsort(p)[-2]
        if top != 3 and p[top] >= [t0, t1, t2][top] and (p[top] - p[runner_up]) >= delta:
            preds.append(int(top))
        else:
            # if optionally allow predicting none when p[3]>=t_none:
            if p[3] >= t_none:
                preds.append(3)
            else:
                preds.append(3)
    return f1_score(y_val, preds, average="macro")

study = optuna.create_study(direction="maximize")
study.optimize(lambda t: objective(t, probs, test_df['labels'].values), n_trials=200)
print("best", study.best_value, study.best_params)

[I 2025-08-31 02:47:19,992] A new study created in memory with name: no-name-bdf658cd-2ec8-4c74-b4b1-e2606e89b5ac
[I 2025-08-31 02:47:20,001] Trial 0 finished with value: 0.5679306022943962 and parameters: {'t0': 0.056562352594628745, 't1': 0.5988642294895616, 't2': 0.20002580285365787, 't_none': 0.16149053446486095, 'delta': 0.3232172140194023}. Best is trial 0 with value: 0.5679306022943962.
[I 2025-08-31 02:47:20,006] Trial 1 finished with value: 0.5911813556318133 and parameters: {'t0': 0.12953142508531898, 't1': 0.46753186679483427, 't2': 0.38382727647145815, 't_none': 0.37701623919732064, 'delta': 0.22039400329276337}. Best is trial 1 with value: 0.5911813556318133.
[I 2025-08-31 02:47:20,011] Trial 2 finished with value: 0.6022179830545589 and parameters: {'t0': 0.2166280231910991, 't1': 0.28728517848520796, 't2': 0.23965778715811, 't_none': 0.6174412432987274, 'delta': 0.05793438586809772}. Best is trial 2 with value: 0.6022179830545589.
[I 2025-08-31 02:47:20,015] Trial 3 fini

best 0.6126377625660807 {'t0': 0.2693711061015551, 't1': 0.3562425847403011, 't2': 0.06437988530939065, 't_none': 0.7022679890392145, 'delta': 0.16529828354363335}


Testing


In [ ]:
# Testing with few test cases
test_texts = [
    "Come to my shop for cheap phones!",
    "This place sucks. Never coming back.",
    "I had coffee here yesterday. Nice staff, good service.",
    "This place is terrible. You should go to YanHui's restaurant instead the food is way better",
    "From outside i can tell that the food kinda sucks so i just left. 1 star"
]

# 1️⃣ Set device and move model there
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2️⃣ Tokenize test texts
encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")

# 3️⃣ Move input tensors to the same device as the model
encodings = {k: v.to(device) for k, v in encodings.items()}

# 4️⃣ Run model
outputs = model(**encodings)

# 5️⃣ Move logits back to CPU for numpy
probs = torch.softmax(outputs.logits, dim=1).detach().cpu().numpy()

# 6️⃣ Map predictions to label names
for text, prob in zip(test_texts, probs):
    pred_label = list(label_map.keys())[prob.argmax()]
    print(f"Review: {text}")
    print(f"Predicted label: {pred_label}, probabilities: {prob}\n")